<a href="https://colab.research.google.com/github/gabadabaduu/laboratorio_2_AD/blob/main/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.parse import urlparse
import json
import csv
from bs4 import BeautifulSoup
from queue import Queue
import requests

In [ ]:
def is_url_ok_to_follow(url, domain):
    parsed_url = urlparse(url)
    return (
        parsed_url.scheme in ("http", "https") and
        parsed_url.netloc.endswith(domain) and
        "@" not in parsed_url.path and
        parsed_url.path.endswith((".html", ""))
    )

In [ ]:
def go(n:int, dictionary:str, output:str):
    visited_urls = set()
    url_queue = Queue()
    base_url = "https://educacionvirtual.javeriana.edu.co/nuestros-programas-nuevo"
    domain = "educacionvirtual.javeriana.edu.co"
    url_queue.put(base_url)
    courses = []

    while not url_queue.empty() and len(visited_urls) < n:
        url = url_queue.get()
        if url not in visited_urls and is_url_ok_to_follow(url, domain):
            visited_urls.add(url)
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                # Obtener la información de los cursos
                course_elements = soup.find_all('div', class_='course-info')
                for course_element in course_elements:
                    # Extraer los datos del curso
                    course_name = course_element.find('h3').text.strip()
                    course_link = url
                    course_duration = course_element.find('span', class_='duration').text.strip()
                    course_level = course_element.find('span', class_='level').text.strip()
                    course_start_date = course_element.find('span', class_='start-date').text.strip()
                    course_price = course_element.find('span', class_='price').text.strip()

                    # Crear un diccionario con los datos del curso y agregarlo a la lista de cursos
                    course_data = {
                        "id": len(courses) + 1,
                        "nombre": course_name,
                        "enlace": course_link,
                        "duracion": course_duration,
                        "nivel": course_level,
                        "fecha_inicio": course_start_date,
                        "precio": course_price
                    }
                    courses.append(course_data)
            else:
                print(f"Failed to fetch URL: {url}")
                continue

            # Obtener enlaces de la página y agregarlos a la cola
            links = [link.get('href') for link in soup.find_all('a', href=True)]
            for link in links:
                url_queue.put(link)

    # Guardar la lista de cursos en el archivo JSON
    with open(dictionary, 'w') as json_file:
        json.dump(courses, json_file, indent=4)

go(10, "cursos.json", "cursos.csv")

In [22]:
base_url = "https://educacionvirtual.javeriana.edu.co/nuestros-programas-nuevo"
domain = "educacionvirtual.javeriana.edu.co"

# Obtener el contenido HTML de la página
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Función para limpiar una palabra
def clean_word(word):
    # Eliminar signos de puntuación al final de la palabra
    word = re.sub(r'[!.,:]+$', '', word)
    return word.lower()

# Función para procesar el texto y extraer palabras
def process_text(text):
    # Expresión regular para identificar palabras válidas
    word_regex = re.compile(r'^[a-zA-ZáéíóúÁÉÍÓÚñÑ][a-zA-ZáéíóúÁÉÍÓÚñÑ0-9_]*$')
    words = []
    for word in text.split():
        cleaned_word = clean_word(word)
        if word_regex.match(cleaned_word) and len(cleaned_word) > 1:
            words.append(cleaned_word)
    return words

# Función para encontrar todas las palabras en el contenido de un curso
def find_words(course_content):
    words = set()
    for tag in course_content.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
        words.update(process_text(tag.get_text()))
    return words

# Función para construir el índice de palabras para un curso
def build_index(course_id, course_content, index, course_counter):
    words = find_words(course_content)
    for word in words:
        index[f"{course_id}_{course_counter:04d}"] = word
        course_counter += 1
    return index, course_counter



# Función para guardar el índice en un archivo CSV
def save_index_to_csv(index, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter='|')
        for word, course_id in index.items():
            writer.writerow([course_id, word])
